In [1]:
import os
from pathlib import Path
import numpy as np # linear algebra
import pandas as pd 
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SparseAdam,Adam,Adagrad,SGD
#from livelossplot import PlotLosses
from tqdm import tqdm
from copy import deepcopy

In [2]:
COLS = ['user_id', 'movie_id', 'rating', 'timestamp']
train_data = pd.read_csv("/kaggle/input/movielens-100k-dataset/ml-100k/u1.base",sep='\t', names=COLS).drop(columns=['timestamp']).astype(int)
test_data = pd.read_csv("/kaggle/input/movielens-100k-dataset/ml-100k/u1.test",sep='\t', names=COLS).drop(columns=['timestamp']).astype(int)
n_users, n_items = 1000,2000
save_basepath = "/kaggle/working/chkpt/"

In [3]:
train_data

,user_id,movie_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3
...,...,...,...
79995,943,1067,2
79996,943,1074,4
79997,943,1188,3
79998,943,1228,3


In [4]:
train_data.movie_id.nunique()

1650

In [5]:
class Matrixfactorization(nn.Module):
    def __init__(self, n_users, n_items, n_dim=20):
        super().__init__()
        self.user_emb = nn.Embedding( n_users, n_dim)
        self.item_emb = nn.Embedding( n_items, n_dim)
        self.user_emb.weight.data.uniform_(0, 0.05)
        self.item_emb.weight.data.uniform_(0, 0.05)
        
    def forward(self, u, v):
        # print(0, u, v)
        u = self.user_emb(u)
        v = self.item_emb(v)
        # print(u.shape, v.shape)
        out = (u*v).sum(1)   
        # print(1, out.shape)
        return out

In [6]:
def train_epocs(model, train_data, test_data, epochs=10, lr=1e-3, wd=0.0, unsqueeze=False, patience = 5, early_stopping = False):
    if not early_stopping:
        print(f"Early stopping not used")
    else:
        print(f"Early stopping used")
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    avg = []
    states = {}
    history = {}
    loss_list = []
    chkpt_list = []
    best_chkpt = None
    best_loss = None
    for i in tqdm(range(1, epochs+1)):
        model.train()
        for it in range(len(train_data)//BATCH_SIZE):
            #---------------SETUP BATCH DATA-------------
            df = train_data.sample(frac=BATCH_SIZE/len(train_data))
            users = torch.LongTensor(df.user_id.values) # .cuda()
            items = torch.LongTensor(df.movie_id.values) #.cuda()
            ratings = torch.FloatTensor(df.rating.values) #.cuda()
            if unsqueeze:
                ratings = ratings.unsqueeze(1)
            y_hat = model(users, items)
            loss = F.mse_loss(y_hat, ratings)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg.append(loss.item())
        print(f"EPOCH {i} Train loss:",sum(avg)/len(avg))
        avg = []
        state = model.state_dict()
        states[i] = deepcopy(state)
        # validation loss
        val_loss = test_loss(model, test_data)
        val_loss = round(val_loss, 3)
        # perform early stopping
        if early_stopping:
            if not loss_list:
                loss_list = [val_loss]
                chkpt_list = [i]
                best_chkpt = i
                best_loss = val_loss
                continue
            if len(loss_list) == patience:
                if min(loss_list) < val_loss:
                    print(f"Stopping early since loss didn't improve after {patience} epochs")
                    break
                else:
                    print(f"loss improved!")
                    loss_list = [val_loss]
                    chkpt_list = [i]
                    best_chkpt = i
                    best_loss = val_loss
            else:
                if min(loss_list) < val_loss:
                    print(f"loss didn't improve!")
                    loss_list = [val_loss] + loss_list
                    chkpt_list = [i] + chkpt_list
                else:
                    print(f"loss improved!")
                    loss_list = [val_loss]
                    chkpt_list = [i]
                    best_chkpt = i
                    best_loss = val_loss
    if best_chkpt is None:
        best_chkpt = i
        best_loss = val_loss
    print(f"Best loss achieved on {best_chkpt}/{epochs} epoch")
    return states[best_chkpt], best_chkpt, best_loss
                
    

In [7]:
def test_loss(model, test_data, unsqueeze=False):
    model.eval()
    users = torch.LongTensor(test_data.user_id.values) #.cuda()
    items = torch.LongTensor(test_data.movie_id.values) #.cuda()
    ratings = torch.FloatTensor(test_data.rating.values) #.cuda()
    if unsqueeze:
        ratings = ratings.unsqueeze(1)
    y_hat = model(users, items)
    loss = F.mse_loss(y_hat, ratings)
    test_loss = loss.item()
    print("test loss %.3f " % test_loss)
    return test_loss

In [8]:
BATCH_SIZE = 32
model = Matrixfactorization(n_users,n_items)
print(model)
chkpt, best_chkpt_index, best_loss = train_epocs(model, train_data, test_data, epochs=20, lr=0.1, patience = 5, early_stopping = True)
save_path = save_basepath + f'mf/' 
Path(save_path).mkdir(parents=True, exist_ok=True)

Matrixfactorization(
  (user_emb): Embedding(1000, 20)
  (item_emb): Embedding(2000, 20)
)
Early stopping used


  5%|▌         | 1/20 [00:08<02:50,  8.97s/it]

EPOCH 1 Train loss: 105.10709141025544
test loss 65.351 


 10%|█         | 2/20 [00:17<02:35,  8.64s/it]

EPOCH 2 Train loss: 84.60182070960998
test loss 74.495 
loss didn't improve!


...

EPOCH 8 Train loss: 85.06658935890198
test loss 70.109 
loss didn't improve!


 45%|████▌     | 9/20 [01:17<01:34,  8.58s/it]

EPOCH 9 Train loss: 88.21413794174194
test loss 77.406 
loss didn't improve!


 45%|████▌     | 9/20 [01:26<01:45,  9.56s/it]

EPOCH 10 Train loss: 89.20692593154907
test loss 70.453 
Stopping early since loss didn't improve after 5 epochs
Best loss achieved on 5/20 epoch


In [9]:
# Saving chkpts
chkpt_path = save_path + f'{best_chkpt_index}_{best_loss}.chkpt'
torch.save(chkpt, chkpt_path)

In [10]:
# Loading chkpt
model2 = Matrixfactorization(n_users,n_items)
model2.load_state_dict(torch.load(chkpt_path, weights_only=True))
test_loss(model2, test_data, )

test loss 64.632 


64.63159942626953